# [Coqui TTS](https://github.com/coqui-ai/TTS) on CPU Real-Time Speech Synthesis 

## Glow-TTS
Paper: https://arxiv.org/abs/2005.11129

Trained with **LJSpeech** for **330K steps**.

## MultiBand-MelGAN
Paper: https://arxiv.org/abs/2005.05106 

In [3]:
import os.path
from os import path

### Download Models

In [4]:
if not path.exists("tts_model.pth.tar"):
    !gdown --id 1NFsfhH8W8AgcfJ-BsL8CYAwQfZ5k4T-n -O tts_model.pth.tar
if not path.exists("config.json"):
    !gdown --id 1IAROF3yy9qTK43vG_-R67y3Py9yYbD6t -O config.json

In [2]:
if not path.exists("tts_model.pth.tar"):
    !gdown --id 1Ty5DZdOc0F7OTGj9oJThYbL5iVu_2G0K -O vocoder_model.pth.tar
if not path.exists("tts_model.pth.tar"):
    !gdown --id 1Rd0R_nRCrbjEdpOwq6XwZAktvugiBvmu -O config_vocoder.json
if not path.exists("tts_model.pth.tar"):
    !gdown --id 11oY3Tv0kQtxK_JPgxrfesa99maVXHNxU -O scale_stats_vocoder.npy

/home/sarieh/anaconda3/lib/python3.9/site-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Ty5DZdOc0F7OTGj9oJThYbL5iVu_2G0K
To: /home/sarieh/gitkraken/Graduation-project/Experiments/text_to_speach_model/vocoder_model.pth.tar
100%|██████████████████████████████████████| 82.8M/82.8M [00:13<00:00, 6.19MB/s]
/home/sarieh/anaconda3/lib/python3.9/site-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Rd0R_nRCrbjEdpOwq6XwZAktvugiBvmu
To: /home/sarieh/gitkraken/Graduation-project/Experiments/text_to_speach_model/config_vocoder.json
100%|██████████████████████████████████████| 6.76k/6.76k [00:00<00:00, 5.34MB/s]
/home/sarieh

### Setup Libraries

In [3]:
!git clone https://github.com/coqui-ai/TTS TTS_repo

Cloning into 'TTS_repo'...
remote: Enumerating objects: 25976, done.
remote: Counting objects: 100% (2384/2384), done.
remote: Compressing objects: 100% (897/897), done.
remote: Total 25976 (delta 1754), reused 1980 (delta 1450), pack-reused 23592
Receiving objects: 100% (25976/25976), 128.16 MiB | 5.38 MiB/s, done.
Resolving deltas: 100% (18892/18892), done.


In [12]:
%cd TTS_repo
!git checkout 4132240
!pip install -r requirements.txt
!python setup.py develop
%cd ..

/home/sarieh/gitkraken/Graduation-project/Experiments/text_to_speach_model/TTS_repo
HEAD is now at 41322408 Merge branch 'dev' of https://github.com/mozilla/TTS into dev
ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.1 (from versions: 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.8.0rc0, 2.8.0rc1, 2.8.0)
ERROR: No matching distribution found for tensorflow==2.3.1
/home/sarieh/anaconda3/lib/python3.9/site-packages/setuptools/dist.py:717: UserWarning: Usage of dash-separated 'build-lib' will not be supported in future versions. Please use the underscore name 'build_lib' instead
  warnings.warn(
/home/sarieh/anaconda3/lib/python3.9/site-packages/setuptools/dist.py:717: UserWarning: Usage of dash-separated 'build-dir' will not be supported in future versions. Please use the underscore name 'build_dir' instead
  warnings.warn(
running develop
--

### Define TTS function

In [13]:
def interpolate_vocoder_input(scale_factor, spec):
    """Interpolation to tolarate the sampling rate difference
    btw tts model and vocoder"""
    print(" > before interpolation :", spec.shape)
    spec = torch.tensor(spec).unsqueeze(0).unsqueeze(0)
    spec = torch.nn.functional.interpolate(spec, scale_factor=scale_factor, mode='bilinear').squeeze(0)
    print(" > after interpolation :", spec.shape)
    return spec


def tts(model, text, CONFIG, use_cuda, ap, use_gl, figures=True):
    t_1 = time.time()
    # run tts
    target_sr = CONFIG.audio['sample_rate']
    waveform, alignment, mel_spec, mel_postnet_spec, stop_tokens, inputs =\
     synthesis(model,
               text,
               CONFIG,
               use_cuda,
               ap,
               speaker_id,
               None,
               False,
               CONFIG.enable_eos_bos_chars,
               use_gl)
    # run vocoder
    mel_postnet_spec = ap._denormalize(mel_postnet_spec.T).T
    if not use_gl:
        target_sr = VOCODER_CONFIG.audio['sample_rate']
        vocoder_input = ap_vocoder._normalize(mel_postnet_spec.T)
        if scale_factor[1] != 1:
            vocoder_input = interpolate_vocoder_input(scale_factor, vocoder_input)
        else:
            vocoder_input = torch.tensor(vocoder_input).unsqueeze(0)
        waveform = vocoder_model.inference(vocoder_input)
    # format output
    if use_cuda and not use_gl:
        waveform = waveform.cpu()
    if not use_gl:
        waveform = waveform.numpy()
    waveform = waveform.squeeze()
    # compute run-time performance
    rtf = (time.time() - t_1) / (len(waveform) / ap.sample_rate)
    tps = (time.time() - t_1) / len(waveform)
    print(waveform.shape)
    print(" > Run-time: {}".format(time.time() - t_1))
    print(" > Real-time factor: {}".format(rtf))
    print(" > Time per step: {}".format(tps))
    # display audio
    IPython.display.display(IPython.display.Audio(waveform, rate=target_sr))  
    return alignment, mel_postnet_spec, stop_tokens, waveform

### Load Models

In [14]:
import sys
import os
import torch
import time
import IPython

# for some reason TTS installation does not work on Colab
sys.path.append('TTS_repo')

from TTS.utils.io import load_config
from TTS.utils.audio import AudioProcessor
from TTS.tts.utils.generic_utils import setup_model
from TTS.tts.utils.text.symbols import symbols, phonemes, make_symbols
from TTS.tts.utils.synthesis import synthesis
from TTS.tts.utils.io import load_checkpoint

In [15]:
# runtime settings
use_cuda = True

In [16]:
# model paths
TTS_MODEL = "tts_model.pth.tar"
TTS_CONFIG = "config.json"
VOCODER_MODEL = "vocoder_model.pth.tar"
VOCODER_CONFIG = "config_vocoder.json"

In [1]:
# load configs
TTS_CONFIG = load_config(TTS_CONFIG)
VOCODER_CONFIG = load_config(VOCODER_CONFIG)

# TTS_CONFIG.audio['stats_path'] = "./scale_stats.npy"
VOCODER_CONFIG.audio['stats_path'] = "./scale_stats_vocoder.npy"


NameError: name 'load_config' is not defined

In [10]:
# load the audio processor
ap = AudioProcessor(**TTS_CONFIG.audio)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:0
 | > fft_size:1024
 | > power:1.1
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:None
 | > hop_length:256
 | > win_length:1024


In [11]:
# LOAD TTS MODEL
# multi speaker 
speakers = []
speaker_id = None
    
if 'characters' in TTS_CONFIG.keys():
    symbols, phonemes = make_symbols(**TTS_CONFIG.characters)

# load the model
num_chars = len(phonemes) if TTS_CONFIG.use_phonemes else len(symbols)
model = setup_model(num_chars, len(speakers), TTS_CONFIG)      

# load model state
model, _ =  load_checkpoint(model, TTS_MODEL, use_cuda=use_cuda)
model.eval();
model.store_inverse();

 > Using model: glow_tts


ModuleNotFoundError: No module named 'TTS.tts.layers.glow_tts.monotonic_align.core'

In [ ]:
from TTS.vocoder.utils.generic_utils import setup_generator

# LOAD VOCODER MODEL
vocoder_model = setup_generator(VOCODER_CONFIG)
vocoder_model.load_state_dict(torch.load(VOCODER_MODEL, map_location="cpu")["model"])
vocoder_model.remove_weight_norm()
vocoder_model.inference_padding = 0

# scale factor for sampling rate difference
scale_factor = [1,  VOCODER_CONFIG['audio']['sample_rate'] / ap.sample_rate]
print(f"scale_factor: {scale_factor}")

ap_vocoder = AudioProcessor(**VOCODER_CONFIG['audio'])    
if use_cuda:
    vocoder_model.cuda()
vocoder_model.eval();

## Run Inference

In [17]:
model.length_scale = 1.0  # set speed of the speech. 
model.noise_scale = 0.33  # set speech variationd

sentence =  "Bill got in the habit of asking himself “Is that thought true?” and if he wasn’t absolutely certain it was, he just let it go."
align, spec, stop_tokedns, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, use_gl=False, figures=True)

(181760,)
 > Run-time: 0.57666015625
 > Real-time factor: 0.06994764897710001
 > Time per step: 3.1722420957726493e-06


In [18]:
# faster speech
model.length_scale = 0.8  # set speed of the speech. 
model.noise_scale = 0.33  # set speech variationd

sentence =  "Bill got in the habit of asking himself “Is that thought true?” and if he wasn’t absolutely certain it was, he just let it go."
align, spec, stop_tokedns, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, use_gl=False, figures=True)

(148480,)
 > Run-time: 0.5099129676818848
 > Real-time factor: 0.07571215612878059
 > Time per step: 3.4336739316068845e-06


In [19]:
# even more faster speech with less variantion
model.length_scale = 0.6  # set speed of the speech. 
model.noise_scale = 0.01  # set speech variation

sentence =  "Bill got in the habit of asking himself “Is that thought true?” and if he wasn’t absolutely certain it was, he just let it go."
align, spec, stop_tokedns, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, use_gl=False, figures=True)

(116736,)
 > Run-time: 0.4951961040496826
 > Real-time factor: 0.09352107452669818
 > Time per step: 4.241338996380045e-06


In [31]:
model.length_scale = 0.9  # set speed of the speech. 
model.noise_scale = 0.3  # set speech variation

sentence =  "Vitamin D is a group of fat-soluble secosteroids responsible for increasing intestinal absorption of calcium, magnesium, and phosphate, and many other biological effects. In humans, the most important compounds in this group are vitamin D₃ and vitamin"
align, spec, stop_tokedns, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, use_gl=False, figures=True)

(380416,)
 > Run-time: 0.42177367210388184
 > Real-time factor: 0.02444259334145011
 > Time per step: 1.1085139018449617e-06
